# Compact sample of ESP fitting with constraints

## Work flow summarized
To begin with, insert implicit hydrogens with `insertHbyList.py`and perform DFT with gpaw.
The following example starts from an already optimized "all-atom" structure in "system100.traj". 

```

module load gpaw

mpirun -n 20 gpaw-python gpw_from_traj.py -c 6 sandbox/system100.traj \
    sandbox/system100.gpw 2>&1 | tee sandbox/gpw_from_traj.log

mpirun -n 20 gpaw-python esp_from_gpw.py sandbox/system100.gpw \
    sandbox/system100.vHtg.cube sandbox/system100.rho.cube \
    sandbox/system100.rho_pseudo.cube 2>&1 | tee sandbox/esp_from_gpw.log

./aa2ua_cube.py sandbox/system100.pdb sandbox/system100.lean.top \
    sandbox/system100.vHtg.cube sandbox/system100.vHtg_ua.cube \
    2>&1 | tee sandbox/aa2ua_cube.vHtg.log

./aa2ua_cube.py sandbox/system100.pdb sandbox/system100.lean.top \
    sandbox/system100.rho.cube sandbox/system100.rho_ua.cube \
    2>&1 | tee sandbox/aa2ua_cube.rho.log

./horton-esp-cost.sh --sign --esp-infile-cube sandbox/system100.vHtg_ua.cube \
    --dens-infile-cube sandbox/system100.rho_ua.cube \
    --cost-outfile-hdf5 sandbox/system100.cost_ua_neg.h5 \
    2>&1 | tee sandbox/horton-esp-cost-ua-neg.log


./fitESPconstrained.py sandbox/system100.cost_ua_neg.h5 sandbox/system100.pdb \
    sandbox/system100.lean.top sandbox/atoms_in_charge_group.csv \
    sandbox/charge_group_total_charge.csv sandbox/atoms_of_same_charge.csv \
    sandbox/fitted_point_charges.txt sandbox/fitted_point_charges.top \
    --qtot 6.0 --verbose 2>&1 | tee sandbox/esp-fit-constrained.log 
    
```

## Work flow explained
The commands summarized above are explained in the following:
It is recommendable to perform the first (and second due to high memory requirement) command as a job or an interactive session in parallel with mpirun, as displayed below. All input files are assumed to exist in subdirectory `sandbox`.

```
module load gpaw

mpirun -n 20 gpaw-python gpw_from_traj.py -c 6 sandbox/system100.traj \
    sandbox/system100.gpw 2>&1 | tee sandbox/gpw_from_traj.log

mpirun -n 20 gpaw-python esp_from_gpw.py sandbox/system100.gpw sandbox/system100.vHtg.cube \
    sandbox/system100.rho.cube sandbox/system100.rho_pseudo.cube \
    2>&1 | tee sandbox/esp_from_gpw.log
    
```

The `2>&1 | tee xyz.log` appendix logs both errors and standard output to screen and to a .log file. The `vHtg.cube` file contains the system's Hartree potential and `rho.cube` the all-electron density, both discretized on a regular grid in cartesian space.

The .cube files are converted to united-atom by removing the atom entries previously inserted by `insertHbyList.py`. Herefore, the original united-atom .pdb and .top are necessary. he `lean.top` file has all `#include` statements (and corresponding `[ molecules ]` entries, e.g. for solvent and background electrolyte ions) removed as to avoid import failure due to unlocatable files.

```
./aa2ua_cube.py sandbox/system100.pdb sandbox/system100.lean.top \
    sandbox/system100.vHtg.cube sandbox/system100.vHtg_ua.cube \
    2>&1 | tee sandbox/aa2ua_cube.vHtg.log

./aa2ua_cube.py sandbox/system100.pdb sandbox/system100.lean.top \
    sandbox/system100.rho.cube sandbox/system100.rho_ua.cube \
    2>&1 | tee sandbox/aa2ua_cube.rho.log
    
```

Next, construct cost function with Horton either by

```
module purge
module load horton/2.1.0b
horton-esp-cost.py sandbox/system100.vHtg_ua.cube sandbox/system100.cost_ua_neg.h5 \
    --pbc 000 --wdens sandbox/system100.rho_ua.cube --overwrite --sign
module purge
module load gpaw
```

or by using the following wrapper to avoid loading and unloading modules

```
./horton-esp-cost.sh --sign --esp-infile-cube sandbox/system100.vHtg_ua.cube \
    --dens-infile-cube sandbox/system100.rho_ua.cube \
    --cost-outfile-hdf5 sandbox/system100.cost_ua_neg.h5 \
    2>&1 | tee sandbox/horton-esp-cost-ua-neg.log
```
The additional `--sign` option is necessary due to non-standardized sign conventions in cube data. If not set, all charges including total charge, group charges and fitted point charges have to be treated with opposite sign in the following.

Finally, use

```
./fitESPconstrained.py sandbox/system100.cost_ua_neg.h5 sandbox/system100.pdb \
    sandbox/system100.lean.top sandbox/atoms_in_charge_group.csv \
    sandbox/charge_group_total_charge.csv sandbox/atoms_of_same_charge.csv \
    sandbox/fitted_point_charges.txt sandbox/fitted_point_charges.top \
    --qtot 6.0 --verbose 2>&1 | tee sandbox/esp-fit-constrained.log 
```
to obtain fitted point charges in a simple text format ordered by the atoms' ASE indices
within `fitted_point_charges.txt` and assigned to atom name and residue within `sandbox/fitted_point_charges.top`. Use `./fitESPconstrained.py --help` for information on the input files' formats. Alternatively, use the code snippets below to obtain point charges within python directly for further processing.

## From within Python

In [10]:
# for log output in jupyter notebook
%config Application.log_level="INFO"
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [11]:
from fitESPconstrained import fitESPconstrained

In [12]:
help(fitESPconstrained)

Help on function fitESPconstrained in module fitESPconstrained:

fitESPconstrained(infile_pdb, infile_top, infile_cost_h5, infile_atoms_in_cg_csv, infile_cg_charges_csv, infile_atoms_of_same_charge_csv, qtot=0.0, strip_string=':SOL,CL', implicitHbondingPartners={'CD4': 1, 'CD3': 1, 'CA2': 2, 'CA3': 2, 'CB2': 2, 'CB3': 2}, debug=False, outfile_top=None)
    Automizes the whole fitting process from importing Horton's
    cost function over reading constraints from simple text files to
    minimizing, logging and double-checking the results.
    
    Parameters
    ----------
    infile_pdb: str
        PDB file with original (united-atom) molecular structure 
    infile_top: str
        GROMACS topolgy file with original (united-atom) system.
        All #includes shoulb be removed!
    infile_cost_h5: str
        Cost function by HORTON, hdf5 format
    infile_atoms_in_cg_csv: str
        file with atom - charge group assignments in simple 
        "comma separated value" text format, o

In [13]:
#q, lagrange_multiplier, info_df, cg2ase, cg2cgtype, cg2q, sym2ase
q, lagrange_multiplier, info_df, cg2ase, \
    cg2cgtype, cg2q, sym2ase = \
    fitESPconstrained(infile_pdb = 'sandbox/system100.pdb', 
                  infile_top = 'sandbox/system100.lean.top', 
                  infile_cost_h5 = 'sandbox/system100.cost_ua_neg.h5', 
                  infile_atoms_in_cg_csv = 'sandbox/atoms_in_charge_group.csv', 
                  infile_cg_charges_csv = 'sandbox/charge_group_total_charge.csv', 
                  infile_atoms_of_same_charge_csv = 'sandbox/atoms_of_same_charge.csv',
                  qtot = 6.0, strip_string=':SOL,CL', 
                  implicitHbondingPartners = {'CD4':1,'CD3':1,'CA2':2,'CA3':2,'CB2':2,'CB3':2},
                  debug=True)

INFO:root:Adding 1 H-atoms to CD3 (#8)...
INFO:root:bondingPartners [ 9 21]
INFO:root:Atom CD3 already has bonding partners CD4, CB1
INFO:root:Adding H-atom 1CD3 at position [ 23.911579364757202, 25.466622991293512, 12.349999999999998 ]
INFO:root:Adding 1 H-atoms to CD4 (#9)...
INFO:root:bondingPartners [10 12]
INFO:root:Atom CD4 already has bonding partners CD5, CA1
INFO:root:Adding H-atom 1CD4 at position [ 25.312471957818314, 25.072450829115194, 10.489569761035874 ]
INFO:root:Adding 2 H-atoms to CA2 (#15)...
INFO:root:bondingPartners [16]
INFO:root:Atom CA2 already has bonding partners CA3
INFO:root:Adding H-atom 1CA2 at position [ 28.74261935828807, 23.557590675857806, 11.104589188842278 ]
INFO:root:bondingPartners [ 16 105]
INFO:root:Atom CA2 already has bonding partners CA3, 1CA2
INFO:root:Adding H-atom 2CA2 at position [ 28.85271690173004, 23.702220854918263, 9.389826475843604 ]
INFO:root:Adding 2 H-atoms to CA3 (#16)...
INFO:root:bondingPartners [17]
INFO:root:Atom CA3 already 

INFO:root:103 unknowns, 80 equality constraints
INFO:root:A (103, 103): 
 [[ 25.815  26.367  25.498 ...,  13.196  21.763  23.254]
 [ 26.367  27.579  25.743 ...,  13.624  22.762  24.405]
 [ 25.498  25.743  25.8   ...,  12.581  20.537  21.908]
 ..., 
 [ 13.196  13.624  12.581 ...,  19.353  16.352  16.166]
 [ 21.763  22.762  20.537 ...,  16.352  25.319  26.016]
 [ 23.254  24.405  21.908 ...,  16.166  26.016  27.391]]
INFO:root:B (103,): 
 [ 95.161  95.834  93.701  95.63   97.396  93.465  92.422  94.419  91.444
  90.587  93.193  93.137  87.457  86.879  84.861  80.451  77.005  73.444
  71.649  70.076  75.291  88.937  86.868  88.269  84.357  80.695  80.383
  80.047  83.916  76.777  94.241  92.734  95.722  95.333  96.809  97.43
  98.159  95.249  94.89   96.707  97.201  93.342  92.693  91.249  89.558
  84.894  82.053  84.636  78.415  80.928  93.905  94.014  91.538  87.599
  83.503  81.055  77.273  83.694  80.088  96.047  95.628  92.206  90.122
  90.709  92.782  96.338  89.992  91.542  84.881  

INFO:root:103 unknowns, 25 equality constraints
INFO:root:A (103, 103): 
 [[ 25.815  26.367  25.498 ...,  13.196  21.763  23.254]
 [ 26.367  27.579  25.743 ...,  13.624  22.762  24.405]
 [ 25.498  25.743  25.8   ...,  12.581  20.537  21.908]
 ..., 
 [ 13.196  13.624  12.581 ...,  19.353  16.352  16.166]
 [ 21.763  22.762  20.537 ...,  16.352  25.319  26.016]
 [ 23.254  24.405  21.908 ...,  16.166  26.016  27.391]]
INFO:root:B (103,): 
 [ 95.161  95.834  93.701  95.63   97.396  93.465  92.422  94.419  91.444
  90.587  93.193  93.137  87.457  86.879  84.861  80.451  77.005  73.444
  71.649  70.076  75.291  88.937  86.868  88.269  84.357  80.695  80.383
  80.047  83.916  76.777  94.241  92.734  95.722  95.333  96.809  97.43
  98.159  95.249  94.89   96.707  97.201  93.342  92.693  91.249  89.558
  84.894  82.053  84.636  78.415  80.928  93.905  94.014  91.538  87.599
  83.503  81.055  77.273  83.694  80.088  96.047  95.628  92.206  90.122
  90.709  92.782  96.338  89.992  91.542  84.881  

INFO:root:
INFO:root:atoms grouped together by their ASE indices:
INFO:root:[array([0, 1, 2]), array([3, 4]), array([ 5,  6,  7,  8,  9, 10, 11]), array([12, 13, 14, 15]), array([16, 17, 18, 19, 20]), array([21, 22, 23, 24]), array([25, 26, 27, 28, 29]), array([30, 31]), array([32, 33]), array([34, 35, 36, 37, 38, 39, 40]), array([41, 42, 43, 44]), array([45, 46, 47, 48, 49]), array([50, 51, 52, 53]), array([54, 55, 56, 57, 58]), array([59, 60]), array([74, 75]), array([76, 77, 78, 79, 80, 81, 82]), array([83, 84, 85, 86]), array([87, 88, 89, 90, 91]), array([92, 93, 94, 95]), array([ 96,  97,  98,  99, 100]), array([101, 102]), array([61, 62]), array([63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73])]
INFO:root:
INFO:root:desired charge of each group:
INFO:root:[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
INFO:root:cg 0, type 1:
INFO:root:                                 q:  0.0000    absolute error:  1.3600e-14
INFO:root:                   q_unconstrained: -0.0

INFO:root:                   q_unconstrained:  0.7899    absolute error: -2.1010e-01
INFO:root:                q_qtot_constrained:  0.8204    absolute error: -1.7963e-01
INFO:root:             q_cg_qtot_constrained:  1.0000    absolute error: -8.8818e-16
INFO:root:cg 21, type 8:
INFO:root:                                 q:  0.0000    absolute error:  1.0797e-14
INFO:root:                   q_unconstrained: -0.3217    absolute error: -3.2174e-01
INFO:root:                q_qtot_constrained: -0.3198    absolute error: -3.1984e-01
INFO:root:             q_cg_qtot_constrained:  0.0000    absolute error:  0.0000e+00
INFO:root:cg 22, type 9:
INFO:root:                                 q:  0.0000    absolute error:  4.3632e-14
INFO:root:                   q_unconstrained:  0.0251    absolute error:  2.5104e-02
INFO:root:                q_qtot_constrained:  0.0085    absolute error:  8.5147e-03
INFO:root:             q_cg_qtot_constrained:  0.0000    absolute error:  4.4409e-16
INFO:root:cg 23

In [14]:
q

array([-0.524,  0.196,  0.328, -0.241,  0.241,  0.843, -0.008, -0.63 ,
       -0.285, -0.169,  0.121,  0.127,  0.799, -0.484, -0.546,  0.231,
        0.584, -1.059,  0.505,  0.42 ,  0.551,  0.799, -0.484, -0.546,
        0.231,  0.584, -1.059,  0.505,  0.42 ,  0.551, -0.241,  0.241,
       -0.241,  0.241, -0.163,  0.299, -0.252,  0.193, -0.224, -0.091,
        0.238,  0.799, -0.484, -0.546,  0.231,  0.584, -1.059,  0.505,
        0.42 ,  0.551,  0.799, -0.484, -0.546,  0.231,  0.584, -1.059,
        0.505,  0.42 ,  0.551, -0.241,  0.241, -0.271,  0.271, -0.986,
        1.051, -0.395, -0.51 ,  0.029,  0.239,  0.223, -0.86 ,  0.445,
        0.81 , -0.045, -0.241,  0.241,  0.257,  0.129, -0.631, -0.411,
       -0.359,  1.329, -0.314,  0.799, -0.484, -0.546,  0.231,  0.584,
       -1.059,  0.505,  0.42 ,  0.551,  0.799, -0.484, -0.546,  0.231,
        0.584, -1.059,  0.505,  0.42 ,  0.551, -0.241,  0.241])

In [15]:
lagrange_multiplier

array([  -9.189, -185.534,   -9.203,   56.143,  -52.699,  -19.233,
         33.006,   79.511,  194.711,   -9.281,   -9.99 ,  -42.738,
        -21.089,    7.76 ,  -40.747, -360.835,   -9.237,  -37.421,
         35.95 ,  -24.132,  -36.879,  257.437,   -9.142,   -9.097,
         88.743,  203.989,  -31.459, -351.581,  266.645,   88.703,
        204.014,  -31.428, -351.61 ,  266.667,   -9.958,   -0.729,
        -11.911,  -28.073,  -14.716,   -9.937,   -0.815,  -12.003,
        -27.958,  -14.534,  -10.014,   -0.741,  -11.952,  -28.162,
        -14.738,  -10.076,   -0.77 ,  -12.202,  -28.144,  -14.484,
         42.123,  -33.712,   16.519,   44.987,  -27.105,   41.864,
        -33.861,   16.526,   44.936,  -26.8  ,   41.749,  -33.737,
         16.358,   44.738,  -26.421,   41.897,  -34.128,   16.639,
         45.061,  -26.947,   41.769,  -33.809,   16.581,   44.975,
        -26.807,    4.599])

## Double-check results
following snippets are samples on how to check the results

### charge group constraints

In [21]:
info_df.iloc[cg2ase[0]] # select first charge group
# q is the fully constraine fit
# q_qtot_constrained is the fit for only the system's total charge constrained
# q_cg_qtot_constrained is the fit for charge groups and total charge constrained, but not symmetries

,atom,residue,q,q_unconstrained,q_qtot_constrained,q_cg_qtot_constrained
0,CE1,terB,-0.524109,-0.329952,-0.324445,-0.307010
1,HE1,terB,0.195820,0.137402,0.128249,0.162811
2,HE2,terB,0.328289,0.146113,0.159843,0.144199


In [22]:
info_df.iloc[cg2ase[0]]['q'].sum() # total charge in first charge group

1.3600232051658168e-14

### symmetry constraints

In [23]:
info_df.iloc[sym2ase[0]] # select first symmetry group

,atom,residue,q,q_unconstrained,q_qtot_constrained,q_cg_qtot_constrained
3,CD1,terB,-0.241387,-0.088691,0.006398,-0.409410
30,CD6,terB,-0.241387,-0.306762,-0.179734,-0.233661
32,CD1,OXO0,-0.241387,-0.189020,-0.371108,-0.153185
59,CD6,OXO0,-0.241387,-0.031134,-0.027982,-0.177481
74,CD1,terA,-0.241387,-0.369452,-0.538505,-0.179503
101,CD6,terA,-0.241387,-0.455664,-0.429103,-0.138428


### total charge of system based upon GPAW electron density

In [30]:
from ase.io.cube import read_cube_data
from ase.units import Bohr
import numpy as np

In [31]:
cube_data, cube_atoms = read_cube_data("sandbox/system100.rho.cube")

In [32]:
unit_cell = cube_atoms.cell.diagonal() / cube_data.shape
unit_volume = np.prod(unit_cell)
q_el = cube_data.sum()*unit_volume/Bohr**3 # integrate electron density
q_core_total = 0
for a in cube_atoms: # count positive core charges
    q_core_total += a.number

In [33]:
q_core_total

494

In [34]:
q_el

488.00318759870044

In [36]:
q_tot = q_core_total - q_el

In [38]:
q_tot # conclusion: integral over electron density correctly reproduces total charge of 6

5.9968124012995645